In [36]:
#importações das bibliotecas
import pandas as pd
import requests

## Campeonato brasileiro 2024

In [ ]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A')

# Informações do campeonato de 2024
tabelas2024 = pd.read_html(seriaA2024.text)
tabela_classificação2024 = tabelas2024[6]
tabela_jogos2024 = tabelas2024[7]
display(tabela_jogos2024)

In [38]:
#criando uma lista com os nomes dos times
nomes_times = list(tabela_jogos2024["Casa \ Fora"])
#criando uma lista com as siglas dos times
sigla_times = list(tabela_jogos2024.columns)
sigla_times.pop(0)

#criando um dicionário com a sigla dos times
apelidos_times = dict(zip(sigla_times, nomes_times))


In [ ]:
# Ajustar a tabela de jogos
tabela_jogos2024_ajustada = tabela_jogos2024.set_index("Casa \ Fora")
# Desempilhar a tabela
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.unstack().reset_index()
# Renomear as colunas
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

# Função para ajustar o nome dos times
def ajustar_apelido(linha):
    time = linha["Fora"]
    return apelidos_times[time]

# Ajustar o nome dos times aplicando a função
tabela_jogos2024_ajustada["Fora"] = tabela_jogos2024_ajustada.apply(ajustar_apelido, axis=1)
# Remover jogos entre o mesmo time
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Fora"] != tabela_jogos2024_ajustada["Casa"]]


In [53]:
tabela_jogos2024_ajustada["Placar"] = tabela_jogos2024_ajustada["Placar"].fillna("A jogar")

jogos_realizados2024 = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Placar"].str.contains("–")]
display(jogos_realizados2024)

jogos_faltantes2024 = tabela_jogos2024_ajustada[~tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes2024 = jogos_faltantes2024.drop(columns=["Placar"])


,Fora,Casa,Placar
11,Athletico Paranaense,Fortaleza,1–0
12,Athletico Paranaense,Grêmio,2–0
14,Athletico Paranaense,Juventude,1–1
15,Athletico Paranaense,Palmeiras,0–2
24,Atlético Goianiense,Botafogo,1–0
...,...,...,...
367,Vitória,Cruzeiro,3–1
379,Vitória,Vasco da Gama,2–1
380,Vasco da Gama,Athletico Paranaense,1–0
390,Vasco da Gama,Fluminense,2–1
